Table 8: Alcohol-specific Geography-specific death rates per 100,000 population, deaths registered in Regions of England, 2001 to 2016

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/causesofdeath/datasets/alcoholspecificdeathsintheukmaindataset')
    sheet = scraper.distribution().as_databaker()

In [2]:
tab = next(tab for tab in sheet if tab.name=='Table 8')

In [3]:
observations = tab.excel_ref('D14').expand(DOWN).expand(RIGHT).is_not_blank()

In [4]:
observations

{<G102 4.3>, <G110 3.2>, <E95 117.0>, <G18 5.8>, <G37 3.2>, <K18 14.2>, <M50 19.8>, <F106 8.6>, <F24 8.6>, <G19 3.4>, <E57 213.0>, <F48 4.8>, <H154 6.1>, <G55 4.0>, <E148 169.0>, <E59 105.0>, <L67 11.2>, <E147 235.0>, <K36 17.9>, <L134 12.3>, <E112 169.0>, <L27 13.9>, <K67 12.6>, <J102 458.0>, <K114 19.6>, <L106 14.6>, <M133 17.5>, <J137 389.0>, <E97 167.0>, <M79 17.8>, <L35 10.7>, <F137 4.5>, <K113 16.1>, <J139 326.0>, <H57 6.1>, <M72 19.2>, <F32 8.8>, <E75 215.0>, <G81 8.3>, <K104 20.4>, <J103 345.0>, <M46 9.9>, <H86 11.2>, <H31 5.7>, <F78 11.3>, <M65 16.0>, <F99 8.5>, <M74 13.0>, <M47 14.2>, <H143 8.3>, <M156 13.0>, <F41 9.0>, <G114 10.0>, <J136 281.0>, <K95 20.7>, <J50 210.0>, <G39 4.1>, <J58 341.0>, <H138 6.2>, <L40 10.0>, <G147 4.7>, <J109 303.0>, <J14 165.0>, <E98 143.0>, <K16 11.5>, <L141 17.5>, <G80 5.7>, <H93 6.4>, <J110 345.0>, <L31 9.8>, <K145 9.3>, <M102 12.7>, <F107 6.5>, <M78 22.3>, <K98 15.5>, <M123 21.2>, <H46 5.5>, <L43 12.9>, <K72 17.5>, <L144 16.3>, <H130 6.6>, <E55

In [5]:
Sex = tab.excel_ref('E12').expand(RIGHT).is_not_whitespace()
Sex

{<J12 'Males'>, <E12 'Females'>}

In [6]:
deaths = tab.excel_ref('E13').expand(RIGHT).is_not_blank()
deaths

{<H13 'Upper 95% confidence limit'>, <J13 'Deaths'>, <K13 'Rate per 100,000 persons1'>, <F13 'Rate per 100,000 persons1'>, <G13 'Lower 95% confidence limit'>, <L13 'Lower 95% confidence limit'>, <E13 'Deaths'>, <M13 'Upper 95% confidence limit'>}

In [7]:
Year = tab.excel_ref('A14').expand(DOWN).is_not_blank() - tab.excel_ref('A320').expand(DOWN)  
Year

{<A50 2005.0>, <A14 2001.0>, <A149 2016.0>, <A113 2012.0>, <A86 2009.0>, <A23 2002.0>, <A32 2003.0>, <A77 2008.0>, <A41 2004.0>, <A95 2010.0>, <A122 2013.0>, <A140 2015.0>, <A59 2006.0>, <A131 2014.0>, <A68 2007.0>, <A104 2011.0>}

In [8]:
Geography = tab.excel_ref('B14').expand(DOWN).is_not_blank() - tab.excel_ref('B158').expand(DOWN)  
Geography

{<B17 'E12000004'>, <B46 'E12000006'>, <B87 'E12000002'>, <B42 'E12000002'>, <B115 'E12000003'>, <B18 'E12000005'>, <B74 'E12000007'>, <B84 'E12000008'>, <B54 'E12000005'>, <B98 'E12000004'>, <B110 'E12000007'>, <B44 'E12000004'>, <B59 'E12000001'>, <B45 'E12000005'>, <B109 'E12000006'>, <B145 'E12000006'>, <B91 'E12000006'>, <B65 'E12000007'>, <B37 'E12000006'>, <B130 'E12000009'>, <B43 'E12000003'>, <B146 'E12000007'>, <B118 'E12000006'>, <B19 'E12000006'>, <B64 'E12000006'>, <B23 'E12000001'>, <B33 'E12000002'>, <B121 'E12000009'>, <B152 'E12000004'>, <B81 'E12000005'>, <B36 'E12000005'>, <B20 'E12000007'>, <B52 'E12000003'>, <B56 'E12000007'>, <B149 'E12000001'>, <B49 'E12000009'>, <B41 'E12000001'>, <B63 'E12000005'>, <B21 'E12000008'>, <B57 'E12000008'>, <B82 'E12000006'>, <B78 'E12000002'>, <B38 'E12000007'>, <B126 'E12000005'>, <B157 'E12000009'>, <B47 'E12000007'>, <B55 'E12000006'>, <B148 'E12000009'>, <B143 'E12000004'>, <B15 'E12000002'>, <B139 'E12000009'>, <B113 'E1200000

In [9]:
Dimensions = [
            HDim(Year,'Year',CLOSEST,ABOVE),
            HDim(Geography,'Geography',DIRECTLY,LEFT),
            HDim(Sex,'Sex',CLOSEST,LEFT),
            HDim(deaths, 'Alcohol Specific Deaths',DIRECTLY,ABOVE),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People')
            ]

In [10]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [11]:
new_table = c1.topandas()
new_table

,OBS,Year,Geography,Sex,Alcohol Specific Deaths,Measure Type,Unit
0,96.0,2001.0,E12000001,Females,Deaths,Count,People
1,7.8,2001.0,E12000001,Females,"Rate per 100,000 persons1",Count,People
2,6.3,2001.0,E12000001,Females,Lower 95% confidence limit,Count,People
3,9.5,2001.0,E12000001,Females,Upper 95% confidence limit,Count,People
4,165.0,2001.0,E12000001,Males,Deaths,Count,People
5,14.3,2001.0,E12000001,Males,"Rate per 100,000 persons1",Count,People
6,12.1,2001.0,E12000001,Males,Lower 95% confidence limit,Count,People
7,16.5,2001.0,E12000001,Males,Upper 95% confidence limit,Count,People
8,278.0,2001.0,E12000002,Females,Deaths,Count,People
9,8.6,2001.0,E12000002,Females,"Rate per 100,000 persons1",Count,People


In [12]:
new_table = new_table[new_table['OBS'] !=  0 ]

In [13]:
new_table['Year'] = pd.to_numeric(new_table['Year'], errors='coerce').fillna(0)

In [14]:
new_table['Year'] = new_table['Year'].astype(int)

In [15]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [16]:
new_table.dtypes

Value                      float64
Year                         int64
Geography                   object
Sex                         object
Alcohol Specific Deaths     object
Measure Type                object
Unit                        object
dtype: object

In [17]:
new_table

,Value,Year,Geography,Sex,Alcohol Specific Deaths,Measure Type,Unit
0,96.0,2001,E12000001,Females,Deaths,Count,People
1,7.8,2001,E12000001,Females,"Rate per 100,000 persons1",Count,People
2,6.3,2001,E12000001,Females,Lower 95% confidence limit,Count,People
3,9.5,2001,E12000001,Females,Upper 95% confidence limit,Count,People
4,165.0,2001,E12000001,Males,Deaths,Count,People
5,14.3,2001,E12000001,Males,"Rate per 100,000 persons1",Count,People
6,12.1,2001,E12000001,Males,Lower 95% confidence limit,Count,People
7,16.5,2001,E12000001,Males,Upper 95% confidence limit,Count,People
8,278.0,2001,E12000002,Females,Deaths,Count,People
9,8.6,2001,E12000002,Females,"Rate per 100,000 persons1",Count,People


In [18]:
nt1 = new_table[new_table['Alcohol Specific Deaths'] == 'Lower 95% confidence limit']

In [19]:
nt1

,Value,Year,Geography,Sex,Alcohol Specific Deaths,Measure Type,Unit
2,6.3,2001,E12000001,Females,Lower 95% confidence limit,Count,People
6,12.1,2001,E12000001,Males,Lower 95% confidence limit,Count,People
10,7.6,2001,E12000002,Females,Lower 95% confidence limit,Count,People
14,15.6,2001,E12000002,Males,Lower 95% confidence limit,Count,People
18,4.1,2001,E12000003,Females,Lower 95% confidence limit,Count,People
22,10.1,2001,E12000003,Males,Lower 95% confidence limit,Count,People
26,4.0,2001,E12000004,Females,Lower 95% confidence limit,Count,People
30,9.2,2001,E12000004,Males,Lower 95% confidence limit,Count,People
34,5.8,2001,E12000005,Females,Lower 95% confidence limit,Count,People
38,12.7,2001,E12000005,Males,Lower 95% confidence limit,Count,People


In [20]:
nt1.columns = ['Lower 95% confidence limit' if x=='Value' else x for x in new_table.columns]

In [21]:
nt2 = new_table[new_table['Alcohol Specific Deaths'] == 'Upper 95% confidence limit']

In [22]:
nt2

,Value,Year,Geography,Sex,Alcohol Specific Deaths,Measure Type,Unit
3,9.5,2001,E12000001,Females,Upper 95% confidence limit,Count,People
7,16.5,2001,E12000001,Males,Upper 95% confidence limit,Count,People
11,9.6,2001,E12000002,Females,Upper 95% confidence limit,Count,People
15,18.6,2001,E12000002,Males,Upper 95% confidence limit,Count,People
19,5.9,2001,E12000003,Females,Upper 95% confidence limit,Count,People
23,12.9,2001,E12000003,Males,Upper 95% confidence limit,Count,People
27,6.1,2001,E12000004,Females,Upper 95% confidence limit,Count,People
31,12.1,2001,E12000004,Males,Upper 95% confidence limit,Count,People
35,7.9,2001,E12000005,Females,Upper 95% confidence limit,Count,People
39,15.7,2001,E12000005,Males,Upper 95% confidence limit,Count,People


In [23]:
nt2.columns = ['Upper 95% confidence limit' if x=='Value' else x for x in new_table.columns]

In [24]:
new_table.count()

Value                      1152
Year                       1152
Geography                  1152
Sex                        1152
Alcohol Specific Deaths    1152
Measure Type               1152
Unit                       1152
dtype: int64

In [25]:
new_table = new_table[new_table['Alcohol Specific Deaths'] != 'Upper 95% confidence limit']

In [26]:
new_table = new_table[new_table['Alcohol Specific Deaths'] != 'Lower 95% confidence limit']

In [27]:
Final_table = pd.merge(new_table, nt1, how = 'inner', on = ['Year','Sex','Geography'])

In [28]:
Final_table.head(2)

,Value,Year,Geography,Sex,Alcohol Specific Deaths_x,Measure Type_x,Unit_x,Lower 95% confidence limit,Alcohol Specific Deaths_y,Measure Type_y,Unit_y
0,96.0,2001,E12000001,Females,Deaths,Count,People,6.3,Lower 95% confidence limit,Count,People
1,7.8,2001,E12000001,Females,"Rate per 100,000 persons1",Count,People,6.3,Lower 95% confidence limit,Count,People


In [29]:
Final_table = Final_table[['Value','Year','Geography','Sex','Alcohol Specific Deaths_x','Measure Type_x','Unit_x',
                           'Lower 95% confidence limit']]

In [30]:
Final_table = pd.merge(Final_table, nt2, how = 'inner', on = ['Year','Sex','Geography'])

In [31]:
Final_table.head(1)

,Value,Year,Geography,Sex,Alcohol Specific Deaths_x,Measure Type_x,Unit_x,Lower 95% confidence limit,Upper 95% confidence limit,Alcohol Specific Deaths,Measure Type,Unit
0,96.0,2001,E12000001,Females,Deaths,Count,People,6.3,9.5,Upper 95% confidence limit,Count,People


In [32]:
Final_table = Final_table[['Year','Sex','Geography','Alcohol Specific Deaths_x','Measure Type','Value','Lower 95% confidence limit', 'Upper 95% confidence limit','Unit']]

In [33]:
Final_table

,Year,Sex,Geography,Alcohol Specific Deaths_x,Measure Type,Value,Lower 95% confidence limit,Upper 95% confidence limit,Unit
0,2001,Females,E12000001,Deaths,Count,96.0,6.3,9.5,People
1,2001,Females,E12000001,"Rate per 100,000 persons1",Count,7.8,6.3,9.5,People
2,2001,Males,E12000001,Deaths,Count,165.0,12.1,16.5,People
3,2001,Males,E12000001,"Rate per 100,000 persons1",Count,14.3,12.1,16.5,People
4,2001,Females,E12000002,Deaths,Count,278.0,7.6,9.6,People
5,2001,Females,E12000002,"Rate per 100,000 persons1",Count,8.6,7.6,9.6,People
6,2001,Males,E12000002,Deaths,Count,517.0,15.6,18.6,People
7,2001,Males,E12000002,"Rate per 100,000 persons1",Count,17.1,15.6,18.6,People
8,2001,Females,E12000003,Deaths,Count,119.0,4.1,5.9,People
9,2001,Females,E12000003,"Rate per 100,000 persons1",Count,5.0,4.1,5.9,People


In [34]:
Final_table.columns = ['Alcohol Specific Deaths' if x=='Alcohol Specific Deaths_x' else x for x in Final_table.columns]

In [35]:
def user_perc(x,y):
    
    if str(x) == 'Deaths':
        return ''
    else:
        return y
    
Final_table['Lower 95% confidence limit'] = Final_table.apply(lambda row: user_perc(row['Alcohol Specific Deaths'],row['Lower 95% confidence limit']), axis = 1)

In [36]:
def user_perc(x,y):
    
    if str(x) == 'Deaths':
        return ''
    else:
        return y
    
Final_table['Upper 95% confidence limit'] = Final_table.apply(lambda row: user_perc(row['Alcohol Specific Deaths'],row['Upper 95% confidence limit']), axis = 1)

In [37]:
Final_table['Alcohol Specific Deaths'] = Final_table['Alcohol Specific Deaths'].str.rstrip('1')

In [38]:
Final_table['Sex'] = Final_table['Sex'].map(
    lambda x: {
        'All persons' : 'T', 
        'Males' : 'M',
        'Females': 'F'         
        }.get(x, x))

In [39]:
Final_table['Age'] =  'All'